In [ ]:
!pip install transformers

In [ ]:
"""!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"""



'!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json\n!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json\n\n!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json\n!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json'

In [ ]:
from transformers import AlbertTokenizerFast, AlbertModel
from transformers import RobertaTokenizerFast, RobertaModel
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import json
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
def compare_models(model1, model2):
  for p1, p2 in zip(model1.parameters(), model2.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        return False
  return True

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('/content/train-v2.0.json.json')
val_contexts, val_questions, val_answers = read_squad('/content/dev-v2.0.json.json')

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
query_model_name = "facebook/dpr-question_encoder-single-nq-base"
passage_model_name = "facebook/dpr-ctx_encoder-single-nq-base"

query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(query_model_name)
passage_tokenizer = DPRContextEncoderTokenizer.from_pretrained(passage_model_name)
query_model = DPRQuestionEncoder.from_pretrained(query_model_name)
passage_model = DPRContextEncoder.from_pretrained(passage_model_name)

query_model.train()
passage_model.train()
print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_query_encodings = query_tokenizer(train_questions, truncation=True, padding=True, return_tensors = 'pt')
train_context_encodings = passage_tokenizer(train_contexts, truncation=True, padding=True, return_tensors = 'pt')

val_query_encodings = query_tokenizer(val_questions, truncation=True, padding=True, return_tensors = 'pt')
val_context_encodings = passage_tokenizer(val_contexts, truncation=True, padding=True, return_tensors = 'pt')

In [ ]:
class DPR(nn.Module):


  def __init__(self, query_model, passage_model, query_tokenizer, passage_tokenizer,
              dense_size, freeze_params = 0.0, batch_size = 2, sample_size = 4):

    '''
    :query_model : The model that encodes queries to dense representation
    :passage_model : The model that encodes passages to dense representation
    :query_tokenizer : tokenizer for queries
    :passage_tokenizer : tokenizer for passages
    :passage_dict : dictionary of passages with their unique id
    :questions : A list of tuples with question and their correct passage id
    :dense_size : the dimension to which the DPR has to encode
    :freeze_params : the percentage of the parameters to be frozen
    :batch_size : the batch size for training
    :sample_size: the sample size for negative sampling
    '''
    super(DPR, self).__init__()
    self.query_model = query_model
    self.query_tokenizer = query_tokenizer
    self.passage_model = passage_model
    self.passage_tokenizer = passage_tokenizer
    self.freeze_params = freeze_params
    self.sample_size = sample_size
    self.batch_size = batch_size

    self.passage_to_dense = nn.Sequential(nn.Linear(768, dense_size * 2),
                                          nn.ReLU(),
                                          nn.Linear(dense_size * 2, dense_size),
                                          nn.GELU())

    self.query_to_dense = nn.Sequential(nn.Linear(768, dense_size * 2),
                                          nn.ReLU(),
                                          nn.Linear(dense_size * 2, dense_size),
                                          nn.GELU())
    self.log_softmax = nn.LogSoftmax(dim = 1)
    self.freeze_layers()


  # Freeze the first self.freeze_params % layers
  def freeze_layers(self):
    num_query_layers = sum(1 for _ in self.query_model.parameters())
    num_passage_layers = sum(1 for _ in self.passage_model.parameters())

    for parameters in list(self.query_model.parameters())[:int(self.freeze_params * num_query_layers)]:
      parameters.requires_grad = False

    for parameters in list(self.query_model.parameters())[int(self.freeze_params * num_query_layers):]:
      parameters.requires_grad = True

    for parameters in list(self.passage_model.parameters())[:int(self.freeze_params * num_passage_layers)]:
      parameters.requires_grad = False

    for parameters in list(self.passage_model.parameters())[int(self.freeze_params * num_passage_layers):]:
      parameters.requires_grad = True

  def get_passage_vectors(self, passage):
    p_vector = self.passage_model(input_ids = passage.input_ids,
                                  attention_mask = passage.attention_mask)
    p_vector = self.query_to_dense(p_vector.pooler_output)
    return p_vector

  def get_query_vector(self, query):
    q_vector = self.query_model(input_ids = query.input_ids,
                                attention_mask = query.attention_mask)
    q_vector = self.query_to_dense(q_vector.pooler_output)
    return q_vector

  def dot_product(self, q_vector, p_vector):
    q_vector = q_vector.unsqueeze(1)
    sim = torch.matmul(q_vector, torch.transpose(p_vector, -2, -1))
    return sim

  def forward(self, context_input_ids, context_attention_mask, query_input_ids, query_attention_mask):
    dense_passage = self.passage_model(input_ids = context_input_ids, attention_mask = context_attention_mask)
    dense_query = self.query_model(input_ids = query_input_ids, attention_mask = query_attention_mask)
    dense_passage = dense_passage['pooler_output']
    dense_query = dense_query['pooler_output']
    dense_passage = self.passage_to_dense(dense_passage)
    dense_query = self.query_to_dense(dense_query)
    similarity_score = self.dot_product(dense_query, dense_passage)
    similarity_score = similarity_score.squeeze(1)
    logits = self.log_softmax(similarity_score)
    return logits

In [ ]:
dpr_model = DPR(query_model = query_model,
                passage_model = passage_model,
                query_tokenizer = query_tokenizer,
                passage_tokenizer = passage_tokenizer,
                dense_size = 64,
                freeze_params = 0.3,
                batch_size = 2,
                sample_size = 8)

dpr_model.train()


sum(p.numel() for p in dpr_model.parameters()), sum(p.numel() for p in dpr_model.parameters() if p.requires_grad == True)

(217996672, 124251520)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(dpr_model.parameters(), lr = 5e-5)

In [ ]:
batch_size = 1
neg_samples = 2
num_questions = len(train_context_encodings.input_ids)

In [ ]:
def get_batch():
  true = []
  context_input_ids_tensor = []
  context_attention_mask_tensor = []
  optimizer.zero_grad()

  true.append(1)

  ## Selecting the query and its positive context
  idx = random.randint(0, num_questions)
  context_input_ids = train_context_encodings.input_ids[idx]
  context_attention_mask = train_context_encodings.attention_mask[idx]
  query_input_ids = train_query_encodings.input_ids[idx]
  query_attention_mask = train_query_encodings.attention_mask[idx]
  context_input_ids_tensor.append(context_input_ids)
  context_attention_mask_tensor.append(context_attention_mask)

  ## Selecting the negative contexts which is hardcoded as 100 index from the selected positive context

  for j in range(neg_samples):
    true.append(0)
    neg_idx_1 = idx + 100
    neg_idx_2 = idx - 100
    if neg_idx_1 < num_questions:
      context_input_ids = train_context_encodings.input_ids[neg_idx_1]
      context_attention_mask = train_context_encodings.attention_mask[neg_idx_1]
      context_input_ids_tensor.append(context_input_ids)
      context_attention_mask_tensor.append(context_attention_mask)

    elif neg_idx_2 > 0:
      context_input_ids = train_context_encodings.input_ids[neg_idx_2]
      context_attention_mask = train_context_encodings.attention_mask[neg_idx_2]
      context_input_ids_tensor.append(context_input_ids)
      context_attention_mask_tensor.append(context_attention_mask)


  context_input_ids_tensor = torch.stack(context_input_ids_tensor)
  context_attention_mask_tensor = torch.stack(context_attention_mask_tensor)
  query_input_ids = query_input_ids.unsqueeze(0)
  query_attention_mask = query_attention_mask.unsqueeze(0)
  return context_input_ids_tensor, context_attention_mask_tensor, query_input_ids, query_attention_mask, true

In [ ]:
batch_loss = 0
for i in range(1, 200):
  context_input_ids_tensor, context_attention_mask_tensor,query_input_ids,query_attention_mask, true = get_batch()
  pred = dpr_model(context_input_ids_tensor, context_attention_mask_tensor,query_input_ids,query_attention_mask)

  true = torch.tensor([0])
  #print(pred.size(), true.size())
  loss = criterion(pred, true)
  loss.backward()
  batch_loss += loss.item()
  optimizer.step()
  if i%20 == 0:
    print(f"Batch : {int(i/20)}  Loss : {batch_loss/20}")
    batch_loss = 0

Batch : 1  Loss : 1.108588308095932
Batch : 2  Loss : 1.1227595925331115
Batch : 3  Loss : 1.0999115526676178
Batch : 4  Loss : 1.1016823530197144
Batch : 5  Loss : 1.0991614639759064
Batch : 6  Loss : 1.0911126017570496
Batch : 7  Loss : 1.0977549791336059
Batch : 8  Loss : 1.1051550149917602
Batch : 9  Loss : 1.10251145362854


In [ ]:
"""save_dir = "/content/drive/MyDrive/Classroom/dpr_3.pt"
torch.save(dpr_model, save_dir)"""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


'from transformers import AutoTokenizer\n\n# Replace "tokenizer_name" with the name of the tokenizer you want to use\ntokenizer = AutoTokenizer.from_pretrained("tokenizer_name")\n\n# Access the configuration\nconfig = tokenizer.config\n\n# Save the tokenizer\'s configuration to a file\nconfig.save_pretrained("/content/drive/MyDrive/Classroom/tokenizer_config.json")'

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

# Example: Initialize and train a Dense Passage Retrieval model
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Save the model and tokenizer
model.save_pretrained("/content/drive/MyDrive/final_dpr")
tokenizer.save_pretrained("/content/drive/MyDrive/final_dpr")

# Save additional metadata or configuration
# For instance, you can save hyperparameters, training data used, etc.

# Example:
metadata = {
    "hyperparameters": {
        "batch_size": 32,
        "learning_rate": 0.001,
        # Add more hyperparameters as needed
    },
    "training_data": "/content/drive/MyDrive/final_dpr",
    # Add any other relevant information
}

# Save metadata to a file
import json
with open("/content/drive/MyDrive/final_dpr.json", "w") as f:
    json.dump(metadata, f)


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained DPR model and tokenizer
model_name = "/content/drive/MyDrive/final_dpr"
model = DPRContextEncoder.from_pretrained(model_name)
tokenizer = DPRContextEncoderTokenizer.from_pretrained(model_name)

# Define your question
question = "Highlight the parts (if any) of this contract related to \"Expiration Date\" that should be reviewed by a lawyer. Details: On what date will the contract's initial term expire?"

# Read passages from a text file
passages_file = "/content/your_text_file.txt"
with open(passages_file, "r", encoding="utf-8") as file:
    passages = file.readlines()

# Encode question and passages
inputs = tokenizer(question, passages, padding=True, truncation=True, return_tensors="pt")

# Pass encoded inputs through the model
with torch.no_grad():
    question_embedding = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]).pooler_output
    passage_embeddings = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], output_hidden_states=True).hidden_states[-1][:, 0, :]

# Compute similarity scores
similarity_scores = cosine_similarity(question_embedding.cpu(), passage_embeddings.cpu())

# Rank passages based on similarity scores
ranked_passages = sorted(zip(passages, similarity_scores[0]), key=lambda x: x[1], reverse=True)

# Retrieve top passages and their similarity scores
top_passages = ranked_passages[:3]  # Retrieving top 3 passages
print("Top passages:")
for passage, score in top_passages:
    print(f"Passage: {passage.strip()}\nSimilarity Score: {score}\n")


Top passages:
Passage: "context": "Exhibit 10.26    CONFIDENTIAL TREATMENT HAS BEEN REQUESTED AS TO CERTAIN PORTIONS OF THIS DOCUMENT. EACH SUCH PORTION,  WHICH HAS BEEN OMITTED HEREIN AND REPLACED WITH AN ASTERISK [*], HAS BEEN FILED SEPARATELY WITH THE  SECURITIES AND EXCHANGE COMMISSION.     PROMOTION AND DISTRIBUTION AGREEMENT     This Promotion and Distribution Agreement including all exhibits (collectively referred to as the \"Agreement\"), effective as of 1 August 2011 (the  \"Effective Date\"), is made by and between Whitesmoke Inc., with registered offices/principle place of business at 501 Silverside Road, Suite 105,  Wilmington DE 19809, USA, (\"Distributor\"), and Google Inc whose principle place of business is at 1600 Amphitheatre Parkway, Mountain View,  CA 94043, USA (\"Google\").\n\n\n\n   \"Bundle\" means the Distribution Products bundled with the Distributor App(s).     \"Chrome Browser\" means the machine-readable binary code version of the Google Chrome browser prov

In [ ]:
import json
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DebertaTokenizer, DebertaForQuestionAnswering
import torch

# Load DPR model and tokenizer for passage retrieval
dpr_model_name = "/content/drive/MyDrive/final_dpr"
dpr_model = DPRContextEncoder.from_pretrained(dpr_model_name)
dpr_tokenizer = DPRContextEncoderTokenizer.from_pretrained(dpr_model_name)

# Load DeBERTa model and tokenizer for answer extraction
deberta_model_name = "/content/drive/MyDrive/deberta v3 base with 100/deberta-v3-base"
deberta_tokenizer = DebertaTokenizer.from_pretrained(deberta_model_name)

"""deberta_model_name = "/content/drive/MyDrive/deberta v3 base with 100/deberta-v3-base"
deberta_model = DebertaForQuestionAnswering.from_pretrained(deberta_model_name, ignore_mismatched_sizes=True)
deberta_tokenizer = DebertaTokenizer.from_pretrained(deberta_model_name)"""

# Load questions and passages from JSON file
with open("/content/dpr_50_CUADv1.json", "r") as file:
    data = json.load(file)

# Retrieve relevant passages using DPR and extract answers using DeBERTa for each question
for question_entry in data["questions"]:
    question = question_entry["question"]
    passages = question_entry["passages"]

    # Retrieve relevant passages using DPR
    def retrieve_passages(question, num_passages=5):
        inputs = dpr_tokenizer(question, return_tensors="pt")
        with torch.no_grad():
            outputs = dpr_model(**inputs)
        passage_ids = outputs["context_input_ids"][:, :num_passages]
        passages = [dpr_tokenizer.decode(passage_id) for passage_id in passage_ids]
        return passages

    passages = retrieve_passages(question)

    # Extract answers from passages using DeBERTa
    def extract_answers(question, passages):
        answers = []
        for passage in passages:
            inputs = deberta_tokenizer(question, passage, return_tensors="pt")
            with torch.no_grad():
                start_logits, end_logits = deberta_model(**inputs)
            start_idx = torch.argmax(start_logits, dim=-1)
            end_idx = torch.argmax(end_logits, dim=-1)
            answer = deberta_tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx+1])
            answers.append(answer)
        return answers

    answers = extract_answers(question, passages)

    # Print the retrieved passages and their answers
    print(f"Question: {question}")
    for passage, answer in zip(passages, answers):
        print(f"Passage: {passage}")
        print(f"Answer: {answer}\n")


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/final_dpr'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
!pip install transformers==4.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
